In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os
import urllib.request
import numpy as np

import PyPDF2
import fitz # PyMuPDF library

In [3]:
# function to get the BS object of an html website.
def get_html(url):
    result = requests.get(url)
    return BeautifulSoup(result.text, "html.parser")

In [143]:
df = pd.DataFrame(columns=["title", "rating", "no_ratings", "time_created", "comments"])

In [4]:
soup = get_html("https://www.chefkoch.de/rezepte/3207211477346212/Schnelles-Spinat-Kichererbsen-Gericht.html")

In [145]:
ids = np.arange(0,100_000)
np.random.shuffle(ids)

In [149]:
user_ids = {}

def parse_recipe_website(page_html, running_id = 0):

    article = page_html.find(class_="ds-container rds").find("article")

    date_raw = article.find(class_="recipe-date rds-recipe-meta__badge").text
    date_created = re.findall(r"\d{2}\.\d{2}\.\d{4}", date_raw)[0]

    title = article.find("h1").text
    avg_rating = float(page_html.find(class_="ds-rating-avg").find("strong").text)
    rating_count = int(page_html.find(class_="ds-rating-count").find("span").find_all("span")[1].text)

    comments = page_html.find_all(class_="ds-mb comment-item qa-comment")
    comments_parsed = {}

    i = running_id

    for comment in comments:
        comment_text = comment.find("p").text
        user = comment.find(class_="ds-h4").find("a").text.strip()

        if user in user_ids:
            comments_parsed[user_ids[user]].append(comment_text)
        else:
            user_ids[user] = ids[i]
            i += 1
            comments_parsed[user_ids[user]] = [comment_text]


    return title, avg_rating, rating_count, date_created, comments_parsed, i

In [150]:
df.append(dict(zip(df.columns, parse_recipe_website(soup))), ignore_index=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17352\923221439.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(dict(zip(df.columns, parse_recipe_website(soup))), ignore_index=True)


,title,rating,no_ratings,time_created,comments
0,Schnelles Spinat-Kichererbsen-Gericht,4.6,500,28.10.2016,"{97547: ['Unglaublich lecker!!!'], 7702: ['Seh..."
